In [1]:
#import libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import CategoricalNB
from sklearn.multioutput import ClassifierChain
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import NearestCentroid
from sklearn.svm import NuSVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OutputCodeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score,classification_report
import sklearn.impute as imputers
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.utils import all_estimators
estimators = all_estimators(type_filter='classifier')
classification_estimators = []
i = 1
for name, class_ in estimators:
    classification_estimators.append(class_.__name__)
print(len(classification_estimators))

41


In [3]:
#load data
df = pd.read_csv('dukecathr_classification_FeatureSelections_for_Imputations.csv')
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
df.head()

,PTNID,AGE_G,GENDER,RACE_G,ACS,CHFSEV,PRE_CULM_CONDITION,HXANGINA,HXCEREB,HXCHF,...,GRAFTST,LADST,LCXST,LMST,LVEF_R,NUMDZV,PRXLADST,RCAST,DEATH,POST_CULM_CONDITION
0,1,7,0,1.0,0,4.0,1,0.0,0,1.0,...,NaN,50.0,25.0,0.0,NaN,1.0,0.0,75.0,1,0
1,2,7,0,1.0,0,NaN,1,1.0,0,NaN,...,100.0,100.0,50.0,0.0,67.439522,2.0,95.0,100.0,0,1
2,3,11,1,1.0,0,0.0,0,1.0,0,0.0,...,NaN,75.0,NaN,0.0,NaN,1.0,0.0,NaN,1,1
3,4,5,0,2.0,1,0.0,1,0.0,0,0.0,...,NaN,0.0,50.0,0.0,56.000000,1.0,0.0,95.0,0,1
4,5,5,0,2.0,0,NaN,1,0.0,0,NaN,...,NaN,0.0,0.0,0.0,51.000000,1.0,0.0,95.0,0,1


In [4]:
# #Splitting the dataset into the Training set and Test set
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [4]:
skf = StratifiedKFold(n_splits = 10, shuffle=True, random_state=42)

indices = skf.split(X, y)
X_train = dict()
y_train = dict()
X_test = dict()
y_test = dict()
cnt = 0
from sklearn.preprocessing import StandardScaler

for train_i, test_i in indices:
    X_train_i = X[train_i]
    X_test_i = X[test_i]
    y_train_i = y[train_i]
    y_test_i = y[test_i]

    sc1 = StandardScaler()
    imputer = imputers.IterativeImputer(random_state=42)
    
    X_train_i = imputer.fit_transform(X_train_i)
    X_train_i = sc1.fit_transform(X_train_i)
    X_test_i = imputer.transform(X_test_i)
    X_test_i = sc1.transform(X_test_i)
    
    X_train[cnt] = X_train_i
    X_test[cnt] = X_test_i
    y_train[cnt] = y_train_i
    y_test[cnt] = y_test_i

    cnt += 1


In [5]:
# #Scaling the Dataset
# from sklearn.preprocessing import StandardScaler
# sc1 = StandardScaler()
# sc2 = StandardScaler()
# imputer = imputers.IterativeImputer(random_state=42)
# # X_train = sc1.fit_transform(X_train)
# X_train = imputer.fit_transform(X_train)
# X_train = sc2.fit_transform(X_train)
# # X_test = sc1.transform(X_test)
# X_test = imputer.transform(X_test)
# X_test = sc2.transform(X_test)

In [6]:
models=[AdaBoostClassifier,BaggingClassifier,BernoulliNB,CalibratedClassifierCV,DecisionTreeClassifier,DummyClassifier,
        ExtraTreeClassifier,ExtraTreesClassifier,GaussianNB,GradientBoostingClassifier,HistGradientBoostingClassifier,KNeighborsClassifier,
        LinearDiscriminantAnalysis,LinearSVC,LogisticRegression,LogisticRegressionCV,MLPClassifier,
        NearestCentroid,PassiveAggressiveClassifier,Perceptron,QuadraticDiscriminantAnalysis,
        RandomForestClassifier,RidgeClassifier,RidgeClassifierCV,SGDClassifier,SVC] 

cross_val_models = dict()
for i in range(10):
        cross_val_models[i] = list()
        for model in models:
                cross_val_models[i].append(model)
            
#CategoricalNB
#ClassifierChain
#ComplementNB
#GaussianProcessClassifier
#LabelPropagation
#LabelSpreading
#MultinomialNB
#NuSVC
#OneVsOneClassifier
#OneVsRestClassifier,
#OutputCodeClassifier
#RadiusNeighborsClassifier
#StackingClassifier
#VotingClassifier

In [7]:
#Fitting the Model
from statistics import mean
from sklearn.metrics import balanced_accuracy_score
accuracy =[]
for i in range(len(models)):
    accuracies = list()
    for k in X_train.keys():
        print("here")
        model = cross_val_models[k][i]()
        model.fit(X_train[k], y_train[k])
        y_pred= model.predict(X_test[k])
        accuracies.append(round(balanced_accuracy_score(y_test[k], y_pred),4))
    accuracy.append(mean(accuracies))
    print (f'model : {models[i]} and  accuracy score is : {mean(accuracies)}')
    
result_df1 = pd.DataFrame({'Accuracy':accuracy}, index=models)
result_df1

here
here
here
here
here
here
here
here
here
here
model : <class 'sklearn.ensemble._weight_boosting.AdaBoostClassifier'> and  accuracy score is : 0.66679
here
here
here
here
here
here
here
here
here
here
model : <class 'sklearn.ensemble._bagging.BaggingClassifier'> and  accuracy score is : 0.70285
here
here
here
here
here
here
here
here
here
here
model : <class 'sklearn.naive_bayes.BernoulliNB'> and  accuracy score is : 0.66211
here
here
here
here
here
here
here
here
here
here
model : <class 'sklearn.calibration.CalibratedClassifierCV'> and  accuracy score is : 0.63779
here
here
here
here
here
here
here
here
here
here
model : <class 'sklearn.tree._classes.DecisionTreeClassifier'> and  accuracy score is : 0.66452
here
here
here
here
here
here
here
here
here
here
model : <class 'sklearn.dummy.DummyClassifier'> and  accuracy score is : 0.5
here
here
here
here
here
here
here
here
here
here
model : <class 'sklearn.tree._classes.ExtraTreeClassifier'> and  accuracy score is : 0.65096
here
her

,Accuracy
<class 'sklearn.ensemble._weight_boosting.AdaBoostClassifier'>,0.66679
<class 'sklearn.ensemble._bagging.BaggingClassifier'>,0.70285
<class 'sklearn.naive_bayes.BernoulliNB'>,0.66211
<class 'sklearn.calibration.CalibratedClassifierCV'>,0.63779
<class 'sklearn.tree._classes.DecisionTreeClassifier'>,0.66452
<class 'sklearn.dummy.DummyClassifier'>,0.50000
<class 'sklearn.tree._classes.ExtraTreeClassifier'>,0.65096
<class 'sklearn.ensemble._forest.ExtraTreesClassifier'>,0.67387
<class 'sklearn.naive_bayes.GaussianNB'>,0.72092
<class 'sklearn.ensemble._gb.GradientBoostingClassifier'>,0.66062
